In [20]:
import xml.etree.ElementTree as ET
import json
# 解析XML文件
tree = ET.parse('laptops/Laptops_Train_v2_Implicit_Labeled.xml')
root = tree.getroot()
data = []
for i in root.findall('sentence'):
    single_data = {'text': i.find('text').text}
    if i.find('aspectTerms'):
        for e in i.find('aspectTerms').findall('aspectTerm'):
            term = e.get('term')
            polarity = e.get('polarity') # negative/positive/neutral/conflict
            implicit = e.get('implicit_sentiment', "####") # False/True
            opinion = e.get('opinion_words', '####') # False/True
            start = e.get('from')
            end = e.get('to')
            if "aspects" in single_data:
                single_data['aspects'].append({'term': term, "polarity": polarity, "implicit": implicit, "from_to": [start, end], "opinion": opinion})
            else:
                single_data['aspects'] = [{'term': term, "polarity": polarity, "implicit": implicit, "from_to": [start, end], "opinion": opinion}]
    data.append(single_data)

file_path = "laptops/laptops_train.json"         # 输出的 JSON 文件路径

# 将 list 写入 JSON 文件
with open(file_path, "w+") as f:
    json.dump(data, f, indent=4)  # indent=4 使 JSON 文件格式化更美观

In [21]:
import json
train_data = []
with open('laptops/laptops_train.json', 'r+', encoding='utf-8') as f:
    system_prompt = """You are a sentiment analysis assistant, performing sentiment analysis on a given laptop product review and corresponding aspects. There are only four emotion categories: positive, negative, conflict, and neutral. The assistant first thinks about the reasoning process in the mind and then provides the user with the answer. The reasoning process and answer are enclosed within <reasoning> </reasoning> and <answer> </answer> tags, respectively, i.e., <reasoning> reasoning process here </reasoning> <answer> answer here </answer>. For example, analyze the sentiment polarity of battery, screen, price and warranty in sentence "Good battery, bad screen, the price is expensive but worth it, not sure about the warranty". <reasoning> Let’s analyze it step by step. The sentence expresses distinct opinions about different aspects. The phrase “Good battery” clearly conveys a positive sentiment toward the battery, using the adjective “good” without any qualifiers. In contrast, “bad screen” directly reflects a negative sentiment about the screen, as “bad” is an unambiguous negative descriptor. The statement “the price is expensive but worth it” presents a conflicted view of the price: while “expensive” suggests a negative perception, “worth it” introduces a compensating positive judgment, indicating mixed feelings. Finally, “not sure about the warranty” reveals uncertainty rather than any clear opinion, making the sentiment toward the warranty neutral. </reasoning> <answer> positive, negative, conflict, neutral </answer>"""
    data = json.load(f)
    for i in data:
        if 'aspects' in i:
            answers = []
            aspects = []
            text = i['text']
            for j in i['aspects']:
                aspects.append(j['term'])
                answers.append(j['polarity'])
#             user_prompt = """<|im_start|>user
# Please analyze the sentiment polarity of aspect {} in sentence "{}"<|im_end|>""".format(', '.join(aspects), ', '.join(answers))
            user_prompt = """Analyze the sentiment polarity of aspect “{}” in sentence “{}”""".format(', '.join(aspects), text)
            train_data.append({"question":user_prompt, "answer":' '.join(answers)})



In [22]:
len(train_data)

1488

In [23]:
with open("laptop_train.json", "w+", encoding='utf-8') as f:
    json.dump(train_data, f, ensure_ascii=False)

In [17]:
a = ["a", "b"]
', '.join(a)

'a, b'